In [1]:
from modules.base import *

#### 스트리밍 사용

- Graph 스트리밍 : 단계별로 출력하는 방식을 의미

- 토큰 스트리밍 : 답변이 토큰 단위로 출력되는것을 의미

---

##### 1. Graph 스트리밍 사용하기 
- 여기서 말하는 스트리밍은 단계별로 출력하는 방식을 의미함. (토큰 스트리밍 X)
- 스트리밍 종류는 2가지

- stream_mode="updates"
    - 각 노드가 실행될때 마다 변경된 state만 업데이트
    - 내생각) `override`
    
- stream_mode="values"
    - 각 노드가 실행될때 마다 전체 state 업데이트
    - 내생각) `append`

In [2]:
@trace_function()
def node_call_model(state: MessagesState, 
               config: RunnableConfig):
    response = llm.invoke(state["messages"], config)
    return {"messages": response}

builder = StateGraph(MessagesState)
builder.add_node("node_call_model", node_call_model)
builder.add_edge(START, "node_call_model")
builder.add_edge("node_call_model", END)
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [3]:
config = {"configurable": {"thread_id": "1"}}
for chunk in graph.stream({"messages": [HumanMessage(content="안녕 나는 창우라고해")]}, 
                          config, 
                          stream_mode="updates"):
    chunk['node_call_model']["messages"].pretty_print()

for chunk in graph.stream({"messages": [HumanMessage(content="나는 올해 30살이고, 현재 부산대학교에서 박사과정을 하고있어.")]}, 
                          config, 
                          stream_mode="updates"):
    chunk['node_call_model']["messages"].pretty_print()
    
for chunk in graph.stream({"messages": [HumanMessage(content="내 취미는 헬스장에서 운동하는거고, 돼지고기를 제일좋아해")]}, 
                          config, 
                          stream_mode="updates"):
    chunk['node_call_model']["messages"].pretty_print()        


🚀 Passing Through [node_call_model] ..
================================== Ai Message ==================================

안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?

🚀 Passing Through [node_call_model] ..
================================== Ai Message ==================================

멋지네요, 창우님! 부산대학교에서 박사과정을 하고 계시다니 대단합니다. 어떤 분야를 연구하고 계신가요? 연구에 대해 이야기해 주시면 흥미롭게 들을게요.

🚀 Passing Through [node_call_model] ..
================================== Ai Message ==================================

운동을 취미로 하신다니 건강을 잘 챙기시는 것 같네요! 헬스장에서 운동을 하면 스트레스 해소에도 좋고 체력도 기를 수 있어서 정말 좋은 취미인 것 같아요. 돼지고기를 좋아하신다니, 혹시 특별히 좋아하는 요리 방법이나 메뉴가 있나요? 예를 들어 삼겹살 구이나 제육볶음 같은 것들이요.


In [4]:
config = {"configurable": {"thread_id": "2"}}
for event in graph.stream({"messages": [HumanMessage(content="안녕 나는 창우라고해")]}, 
                          config, 
                          stream_mode="values"):
    pass

for m in event['messages']:
    m.pretty_print()

print(f"\n{RED}{'*'*80}{RESET}")

for event in graph.stream({"messages": [HumanMessage(content="나는 올해 30살이고, 현재 부산대학교에서 박사과정을 하고있어.")]}, 
                          config, 
                          stream_mode="values"):
    pass
for m in event['messages']:
    m.pretty_print()

print(f"\n{RED}{'*'*80}{RESET}")
    
for event in graph.stream({"messages": [HumanMessage(content="내 취미는 헬스장에서 운동하는거고, 돼지고기를 제일좋아해")]}, 
                          config, 
                          stream_mode="values"):
    pass
for m in event['messages']:
    m.pretty_print()


🚀 Passing Through [node_call_model] ..
================================ Human Message =================================

안녕 나는 창우라고해
================================== Ai Message ==================================

안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?

********************************************************************************

🚀 Passing Through [node_call_model] ..
================================ Human Message =================================

안녕 나는 창우라고해
================================== Ai Message ==================================

안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?
================================ Human Message =================================

나는 올해 30살이고, 현재 부산대학교에서 박사과정을 하고있어.
================================== Ai Message ==================================

정말 멋지네요, 창우님! 부산대학교에서 박사과정을 밟고 계시다니 대단합니다. 어떤 분야에서 공부하고 계신가요? 연구 주제나 관심 분야에 대해 조금 더 알려주시면 좋겠어요.

********************************************************************************

🚀 Passing Through [node_call_model] .

#### 2. 토큰 스트리밍

하나씩 출력

In [5]:
node_to_stream = 'node_call_model'
config = {"configurable": {"thread_id": "4"}}
input_message = HumanMessage(content="이순신 장군에 대해 알려줘")
async for event in graph.astream_events({"messages": [input_message]}, 
                                        config, 
                                        version="v2"):
    if event["event"] == "on_chat_model_stream" and event['metadata'].get('langgraph_node','') == node_to_stream:
        print(event["data"])


🚀 Passing Through [node_call_model] ..
{'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-b86e2ae3-f6da-419f-bf34-3d526f894402')}
{'chunk': AIMessageChunk(content='이', additional_kwargs={}, response_metadata={}, id='run-b86e2ae3-f6da-419f-bf34-3d526f894402')}
{'chunk': AIMessageChunk(content='순', additional_kwargs={}, response_metadata={}, id='run-b86e2ae3-f6da-419f-bf34-3d526f894402')}
{'chunk': AIMessageChunk(content='신', additional_kwargs={}, response_metadata={}, id='run-b86e2ae3-f6da-419f-bf34-3d526f894402')}
{'chunk': AIMessageChunk(content=' 장', additional_kwargs={}, response_metadata={}, id='run-b86e2ae3-f6da-419f-bf34-3d526f894402')}
{'chunk': AIMessageChunk(content='군', additional_kwargs={}, response_metadata={}, id='run-b86e2ae3-f6da-419f-bf34-3d526f894402')}
{'chunk': AIMessageChunk(content='은', additional_kwargs={}, response_metadata={}, id='run-b86e2ae3-f6da-419f-bf34-3d526f894402')}
{'chunk': AIMessageChunk(content=' 조', additional_

이어 붙이기

In [6]:
node_to_stream = 'node_call_model'
config = {"configurable": {"thread_id": "4"}}
input_message = HumanMessage(content="이순신 장군에 대해 알려줘")
async for event in graph.astream_events({"messages": [input_message]}, 
                                        config, 
                                        version="v2"):
    if event["event"] == "on_chat_model_stream" and event['metadata'].get('langgraph_node','') == node_to_stream:
        print(f"{YELLOW}{event['data']['chunk'].content}{RESET}", end="", flush=True)


🚀 Passing Through [node_call_model] ..
이순신 장군은 조선 시대의 대표적인 해군 장군으로, 임진왜란(1592-1598) 동안 조선을 침략한 일본군에 맞서 큰 승리를 이끈 인물입니다. 1545년 서울에서 태어난 그는 무관으로서의 자질을 인정받아 여러 군사적 직책을 수행하였고, 결국 전라좌수사로 임명되어 해군을 지휘하게 되었습니다.

이순신 장군은 특히 해전에서 뛰어난 전략가로서 명성을 얻었습니다. 그의 가장 유명한 전투는 다음과 같습니다:

1. **한산도 대첩 (1592년)**: 이 전투에서 이순신 장군은 학익진 전술을 사용하여 일본 함대를 대파하였습니다. 이 승리는 조선 수군의 사기를 크게 높이고, 제해권을 확보하는 데 중요한 역할을 했습니다.

2. **명량 해전 (1597년)**: 명량 해전은 이순신 장군의 전설적인 승리 중 하나로, 13척의 배로 수백 척의 일본 함대를 물리쳤습니다. 이 전투는 그의 지휘 능력과 전략적 통찰력을 잘 보여줍니다.

3. **노량 해전 (1598년)**: 임진왜란의 마지막 해전으로, 이순신 장군은 이 전투 중에 전사하였으나, 일본군을 크게 무찌르며 전쟁을 마무리 지었습니다.

이순신 장군은 그의 철저한 준비성과 군사적 통찰력, 그리고 부하들과의 신뢰를 바탕으로 한 리더십으로 크게 존경받고 있습니다. 그의 난중일기는 당시의 전쟁 상황과 그의 고뇌, 결단을 생생히 기록한 자료로 남아 있습니다. 이순신 장군은 오늘날에도 한국에서 가장 존경받는 역사적 인물 중 한 명입니다.